<center><h1> Enhanced Index Tracking with CVAR-Based Ratio Measures <h1/><center/>

This notebook tries to replicate the results from the above paper.

## Preamble

In [109]:
import pandas as pd
import numpy as np
from mosek.fusion import *

## Loading Data

In [110]:
pathCSV = "./Data/Data from Guastaroba (2020)/CSV";

Read data for the UU-period.

In [111]:
IS_UU = pd.read_csv(pathCSV + "/IS-GMS-UU.csv", header=None);
IS_UU = IS_UU.values;
OoS_UU = pd.read_csv(pathCSV + "/OoS-GMS-UU.csv", header=None);
OoS_UU = OoS_UU.values;

Read data for the UD-period.

In [112]:
IS_UD = pd.read_csv(pathCSV + "/IS-GMS-UD.csv", header=None);
IS_UD = IS_UD.values;
OoS_UD = pd.read_csv(pathCSV + "/OoS-GMS-UD.csv", header=None);
OoS_UD = OoS_UD.values;

Read data for the DD-period.

In [113]:
IS_DD = pd.read_csv(pathCSV + "/IS-GMS-DD.csv", header=None);
IS_DD = IS_DD.values;
OoS_DD = pd.read_csv(pathCSV + "/OoS-GMS-DD.csv", header=None);
OoS_DD = OoS_DD.values;

Read data for the DU-period.

In [114]:
IS_DU = pd.read_csv(pathCSV + "/IS-GMS-DU.csv", header=None);
IS_DU = IS_DU.values;
OoS_DU = pd.read_csv(pathCSV + "/OoS-GMS-DU.csv", header=None);
OoS_DU = OoS_DU.values;

The indices are located in the first column.

## Building model with MOSEK's Fusion API

We start by specifying necessary constants.

In [115]:
T = 104;
N = 100;
K = 5;

In [116]:
def WCVaR(data, alpha, N, T, K):
    
    eps1 = 0.0001;
    eps2 = 0.0001;
    
    alphaAnnualy = alpha;
    alphaWeekly = ((1 + alphaAnnualy)**(1/52) - 1);
    
    means = np.mean(IS_UU, axis=0);
    mu = means[1:];
    mu_alpha = means[0] + alphaWeekly;
    
    e = np.ones(N);
    
    # Constants for CVaR
    omega = np.array([1/5 for i in range(5)]);
    eta = np.array([0.03, 0.10, 0.15, 0.17, 0.20]);
    
    # Separate input data
    rI = data[:,0];
    rS = data[:,1:];
    
    with Model('WCVaR') as M:
        
        # Define variables
        x = M.variable("x", N, Domain.greaterThan(0.0));
        d = M.variable("d", [T,K], Domain.greaterThan(0.0));
        
        # Define constraints
        M.constraint('first', Expr.sum(Expr.mult(Expr.sub(mu,e*mu_alpha), x)), Domain.equalsTo(1.0));
        M.constraint('second', Expr.sum(x), Domain.lessThan(1/eps1));
        
        for t in range(T):
            for k in range(K):
                M.constraint('c{}{}'.format(t,k), Expr.add(Expr.sub(d.index(t,k), eta[k]), Expr.sum(Expr.sub(rS[t,:],e*rI[t]))), Domain.greaterThan(0.0));
        
        # Define objective
        M.objective('obj', ObjectiveSense.Minimize, Expr.sum(Expr.mult(omega,eta)));
        
        
        

In [97]:
e = np.ones(100)

In [107]:
omega = np.array([1/5 for i in range(5)]);
eta = np.array([0.03, 0.10, 0.15, 0.17, 0.20]);

In [108]:
omega/eta

array([6.66666667, 2.        , 1.33333333, 1.17647059, 1.        ])